In [99]:
import islpy as isl

def codegen(sched):
    bld = isl.AstBuild.from_context(isl.Set("{:}"))
    mp = isl.Map.from_domain_and_range(sched, sched)
    mp = isl.Map.identity(mp.get_space())
    mp = isl.Map.from_domain(sched)
    ast = bld.ast_from_schedule(mp)
    ptr = isl.Printer.to_str(isl.DEFAULT_CONTEXT)
    ptr = ptr.set_output_format(isl.format.C)
    ptr.flush()
    ptr = ptr.print_ast_node(ast)
    return ptr.get_str()

def replaceConsts(code, consts):
    for cname in consts:
        code = re.sub(r"\b%s\b" % cname, str(consts[cname]), code)
    return code

ctx = isl.DEFAULT_CONTEXT

code = "[N_R, m, n] -> { lb_j[i, j] : 0 <= i < N_R && j = 0; ub_j[i, j] : 0 <= i < N_R && j = 0; spmv[i, j] : 0 <= i < N_R && m <= j < n }"
spmv = isl.Set("[N_R,m,n]->{spmv[i,j] : 0 <= i < N_R && m <= j < n}", ctx)
lb_j = isl.Set("[N_R,m,n]->{lb_j[i] : 0 <= i < N_R}", ctx)
ub_j = isl.Set("[N_R,m,n]->{ub_j[i] : 0 <= i < N_R}", ctx)
icsr = lb_j.union(ub_j)#.union(spmv)
print("/* I_csr: %s */" % str(icsr))
print(codegen(icsr))




Error: call to isl_set_union failed: isl_error_invalid

In [93]:
import islpy as isl

def codegen(sched):
    bld = isl.AstBuild.from_context(isl.Set("{:}"))
    mp = isl.Map.from_domain_and_range(sched, sched)
    mp = isl.Map.identity(mp.get_space())
    mp = isl.Map.from_domain(sched)
    ast = bld.ast_from_schedule(mp)
    ptr = isl.Printer.to_str(isl.DEFAULT_CONTEXT)
    ptr = ptr.set_output_format(isl.format.C)
    ptr.flush()
    ptr = ptr.print_ast_node(ast)
    return ptr.get_str()

def replaceConsts(code, consts):
    for cname in consts:
        code = re.sub(r"\b%s\b" % cname, str(consts[cname]), code)
    return code

code = "[N_R, index_i_, index_ip1_] -> { spmv[i, j] : 0 <= i < N_R and index_i_ <= j < index_ip1_ }"

densef = "[N_R, N_C, index_i_, index_ip1_, col_j_] -> {spmv[i,j] -> spmv[i,k,j]: 0 <= k < N_C && k=col_j_}"

#tilef = """[N_R, N_C, index_i_, index_ip1_] -> {spmv[i,k,j] -> [ii,kk,i,k,j]: 0 <= i < N_R && 0 <= k < N_C &&
#            exists ri, ck: 0 <= ri < R && i = ii * R + ri && 0 <= ck < C && k = kk * C + ck &&
#            index_i_ <= j < index_ip1_ && N_R > R && N_C > C}"""

tilef = "[N_R,N_C,index_i_,index_ip1_] -> { spmv[ii,kk,i,k,j] : 0 <= i < N_R && 0 <= k < N_C && exists ri,ck :" + \
    "0 <= ri < R && i=ii*R+ri && 0 <= ck < C && k=kk*C+ck && index_i_ <= j < index_ip1_ && N_C > C && N_R > R}"

consts = {'R':8, 'C':8}
tilef = replaceConsts(tilef, consts)

execf = "[N_R, N_C, R, C, NB, b_index_ii_, b_index_iip1_, b_col_jj_] -> {spmv[ii,kk,i,k,j] -> spmv[ii,jj,i,k,j]: " + \
        "b_index_ii_ <= jj < b_index_iip1_ && kk = b_col_jj_ && j = 0}"

compf = "[N_R,N_C,index_i_,index_ip1_,col_j_] -> {spmv[ii,kk,i,k,j] -> insp[ii,kk,i,k,j]: k = col_j_ && kk = k/C}"
compf = replaceConsts(compf, consts)

ctx = isl.DEFAULT_CONTEXT

bdom = '[N_R, N_C, R, C, NB, b_index_ii_, b_index_iip1_, b_col_jj_, index_i_, index_ip1_] -> { spmv[ii, jj, i, k, 0] : N_R >= 10 and N_C >= 10 and index_i_ <= 0 and index_ip1_ > 0 and b_index_ii_ <= jj < b_index_iip1_ and i >= 9ii and 0 <= i <= 8 + 9ii and i < N_R and k >= 9b_col_jj_ and 0 <= k <= 8 + 9b_col_jj_ and k < N_C }'
ibsr = isl.Set(bdom, ctx)
print("/* I_bsr: %s */" % str(ibsr))
print(codegen(ibsr))

icsr = isl.Set(code, ctx)
print("/* I_csr: %s */" % str(icsr))
print(codegen(icsr))

mkdense = isl.Map(densef, ctx)
print("/* T_md: %s */" % str(mkdense))
idense = icsr.apply(mkdense)
print("/* I_dense: %s */" % str(idense))
print(codegen(idense))

#ttile = isl.Map(tilef, ctx)
#itile = idense.apply(ttile)
itile = isl.Set(tilef, ctx)
print("/* I_tile: %s */" % itile)
print(codegen(itile))

texec = isl.Map(execf, ctx)
iexec = itile.apply(texec)
print("/* I_exec: %s */" % iexec)
print(codegen(iexec))

tcomp = isl.Map(compf, ctx)
icomp = itile.apply(tcomp)
print("/* I_comp: %s */" % icomp)
print(codegen(icomp))


/* I_bsr: [N_R, N_C, R, C, NB, b_index_ii_, b_index_iip1_, b_col_jj_, index_i_, index_ip1_] -> { spmv[ii, jj, i, k, 0] : N_R >= 10 and N_C >= 10 and index_i_ <= 0 and index_ip1_ > 0 and b_index_ii_ <= jj < b_index_iip1_ and i >= 9ii and 0 <= i <= 8 + 9ii and i < N_R and k >= 9b_col_jj_ and 0 <= k <= 8 + 9b_col_jj_ and k < N_C } */
if (N_R >= 10 && N_C >= 10 && b_col_jj_ >= 0 && index_i_ <= 0 && index_ip1_ >= 1)
  for (int c0 = 0; c0 <= (N_R - 1) / 9; c0 += 1)
    for (int c1 = b_index_ii_; c1 < b_index_iip1_; c1 += 1)
      for (int c2 = 9 * c0; c2 <= min(N_R - 1, 9 * c0 + 8); c2 += 1)
        for (int c3 = 9 * b_col_jj_; c3 <= min(N_C - 1, 9 * b_col_jj_ + 8); c3 += 1)
          spmv(c0, c1, c2, c3, 0);

/* I_csr: [N_R, index_i_, index_ip1_] -> { spmv[i, j] : 0 <= i < N_R and index_i_ <= j < index_ip1_ } */
for (int c0 = 0; c0 < N_R; c0 += 1)
  for (int c1 = index_i_; c1 < index_ip1_; c1 += 1)
    spmv(c0, c1);

/* T_md: [N_R, N_C, index_i_, index_ip1_, col_j_] -> { spmv[i, j] -> spmv[

In [1]:
code = """
Icsr1 := [N_R, m, n] -> {lb_j[i]: 0 <= i < N_R; ub_j[i]: 0 <= i < N_R; spmv[i,j] : 0 <= i < N_R && m <= j < n};
Tfuse := [N_R, m, n] -> {
    lb_j[i] -> [0,i,0,0,0];
    ub_j[i] -> [0,i,1,0,1];
    spmv[i,j] -> [0,i,2,j,2];
};
Icsr := Tfuse * Icsr1;
print(Icsr);
codegen(Icsr);

Tdense := [N_R, N_C, m, n] -> {
    spmv[i,j] -> spmv[0,i,2,k,j,2] : 0 <= k < N_C
};

Ismv := [N_R, N_C, m, n] -> {lb_j[i]: 0 <= i < N_R; ub_j[i]: 0 <= i < N_R; spmv_if[i,k,j] : 0 <= i < N_R && 0 <= k < N_C && m <= j < n};
Tdns := [N_R, N_C, m, n] -> {
    lb_j[i] -> [0,i,0,0,0];
    ub_j[i] -> [0,i,1,1,1];
    spmv_if[i,j] -> [0,i,2,k,2,j];
};
Idns := Tdns * Ismv;
print(Idns);
codegen(Idns);
"""

cmd = '/usr/local/bin/iscc'# + ' --help'
import subprocess as sub
proc = sub.Popen(cmd, shell=True, stdin=sub.PIPE, stdout=sub.PIPE, stderr=sub.PIPE)
data = proc.communicate(input=code.encode('ascii'))
out = data[0].decode()
err = data[1].decode()

if len(out) > 0:
    print("Output:")
    print("=========================================================================================================\n")
    print(out)

if len(err) > 0:
    print("Error:")
    print("=========================================================================================================\n")
    print(err)


Output:

[N_R, m, n] -> { lb_j[i] -> [0, i, 0, 0, 0] : 0 <= i < N_R; ub_j[i] -> [0, i, 1, 0, 1] : 0 <= i < N_R; spmv[i, j] -> [0, i, 2, j, 2] : 0 <= i < N_R and m <= j < n }
for (int c1 = 0; c1 < N_R; c1 += 1) {
  lb_j(c1);
  ub_j(c1);
  for (int c3 = m; c3 < n; c3 += 1)
    spmv(c1, c3);
}
[N_R, N_C, m, n] -> { ub_j[i] -> [0, i, 1, 1, 1] : 0 <= i < N_R; lb_j[i] -> [0, i, 0, 0, 0] : 0 <= i < N_R }
for (int c1 = 0; c1 < N_R; c1 += 1) {
  lb_j(c1);
  ub_j(c1);
}



In [81]:
code = """
# ISL
I_csr := [N_R,index_i_,index_ip1_] -> { spmv[i,j] : 0 <= i < N_R and index_i_ <= j < index_ip1_};
print("/* I_csr: ");
print(I_csr);
print(" */");
codegen(I_csr);

T_md := [N_R,N_C,index_i_,index_ip1_] -> { spmv[i,j] -> [i,k,j] : 0 <= i < N_R && 0 <= k < N_C && index_i_ <= j < index_ip1_};
#I_md := [N_R,N_C,index_i_,index_ip1_,col_j_] -> { spmv[i,k,j] : 0 <= i < N_R and 0 <= k < N_C and k=col_j_ and index_i_ <= j < index_ip1_ };
I_md := T_md * I_csr;
print("/* I_md: ");
print(I_md);
print(" */");
codegen(I_md);

# R=C=8
# i=R*ii+ri
#FLUX1Xv[c,z,y,x] -> [tz,ty,tx,0,c,z+1,y+1,x,0] : exists rz,ry,rx: 0<=rz<16 && z+1=tz*16+rz &&
#                                                                  0<=ry<16 && y+1=ty*16+ry &&
#                                                                 -1<=rx<16 && x=tx*16+rx;
#T_tile := [N_R,N_C,index_i_,index_ip1_] -> { spmv[i,k,j] -> [ii,kk,i,k,j] : exists ri,ck :
#    0 <= ri < 8 && i=ii*8+ri && 0 <= ck < 8 && k=kk*8+ck && index_i_ <= j < index_ip1_ && N_C > 8 && N_R > 8};
#I_tile := [N_R,N_C,index_i_,index_ip1_] -> { spmv[ii,kk,ri,ck,j] : 0 <= ii < N_R/8 && 0 <= kk < N_C/8 && 0 <= ri < 8 && 0 <= ck < 8 && index_i_ <= j < index_ip1_ };
#I_tile := T_tile * I_md;
I_tile := [N_R,N_C,index_i_,index_ip1_] -> { spmv[ii,kk,i,k,j] : 0 <= i < N_R && 0 <= k < N_C && exists ri,ck :
    0 <= ri < 8 && i=ii*8+ri && 0 <= ck < 8 && k=kk*8+ck && index_i_ <= j < index_ip1_ && N_C > 8 && N_R > 8};
print("/* I_tile: ");
print(I_tile);
print(" */");
codegen(I_tile);

# PLDI Executor
T_exec := [N_R,N_C,R,C,NB,b_row_ii_,b_row_iip1_,b_col_jj_] -> { spmv[ii,kk,i,k,j] -> [ii,jj,i,k,j] : b_row_ii_ <= jj < b_row_iip1_ && kk=b_col_jj_ && j = 0 };
#I_exec := [N_R,N_C,R,C,NB,block_row_ii_,block_row_iip1_,block_col_jj_] -> { bcsr_spmv2[ii,b,kk,ri,ck] : 0 <= ii < N_R/2 and block_row_ii_ <= b < block_row_iip1_ and kk=block_col_b_ and 0 <= ri < R and 0 <= ck < C };
I_exec := T_exec * I_tile;
print("/* I_exec: ");
print(I_exec);
print(" */");
codegen(I_exec);

# Inspector
T_comp := [N_R,N_C,index_i_,index_ip1_,col_j_] -> {spmv[ii,kk,i,k,j] -> [ii,kk,i,k,j]: k = col_j_ && kk = k/8};
I_comp := T_comp * I_tile;
print("/* I_comp: ");
print(I_comp);
print(" */");
codegen(I_comp);
"""

rest="""
# IMPACT Executor
# Forcing j=0 kinda hacky, but works...
T_comp := [N_R,N_C,R,C,NB,block_row_b_,block_col_b_] -> { spmv[ii,kk,ri,ck,j] -> [b,ii,kk,ri,ck] : 0 <= b < NB and ii=block_row_b_ and kk=block_col_b_ and 0 <= ri < R and 0 <= ck < C and j = 0 };
#I_exec := [N_R,N_C,R,C,NB,block_row_b_,block_col_b_] -> { bsr_spmv[b,ii,kk,ri,ck] : 0 <= b < NB and ii=block_row_b_ and kk=block_col_b_ and 0 <= ri < R and 0 <= ck < C };
I_exec := T_comp * I_tile;
print("/* I_exec: ");
print(I_exec);
print(" */");
codegen(I_exec);

# Inspector
Bset := [N_R,N_C,R,C,col_j_,index_i_,index_ip1_] -> {[ii,ri,i,j] : 0 <= ii < N_R/2 and exists(kk = col_j_/2 : 0 <= col_j_ < N_C) and 0 <= ri < R and i=ii*2+ri and index_i_ <= j < index_ip1_};
print(Bset);
NB := card(Bset);
#c := order(Bset);
R_A_Aprime := [NB,R,C,col_j_] -> {[j] -> [b,ri,ck] : 0 <= b < NB and 0 <= ri < R and 0 <= ck < C and ck=col_j_-col_j_/2*2};
print(R_A_Aprime);

S := [N,F] -> {[i,j]  : 0<=i<N and 0<=j<F};
codegen(S);


"""

#print("Input:")
#print(code);

cmd = '/usr/local/bin/iscc'# + ' --help'
import subprocess as sub
proc = sub.Popen(cmd, shell=True, stdin=sub.PIPE, stdout=sub.PIPE, stderr=sub.PIPE)
data = proc.communicate(input=code.encode('ascii'))
out = data[0].decode()
err = data[1].decode()

if len(out) > 0:
    print("Output:")
    print("=========================================================================================================\n")
    print(out)

if len(err) > 0:
    print("Error:")
    print("=========================================================================================================\n")
    print(err)


Output:

"/* I_csr: "
[N_R, index_i_, index_ip1_] -> { spmv[i, j] : 0 <= i < N_R and index_i_ <= j < index_ip1_ }
" */"
for (int c0 = 0; c0 < N_R; c0 += 1)
  for (int c1 = index_i_; c1 < index_ip1_; c1 += 1)
    spmv(c0, c1);
"/* I_md: "
[N_R, index_i_, index_ip1_, N_C] -> { spmv[i, j] -> [i, k, j] : 0 <= i < N_R and index_i_ <= j < index_ip1_ and 0 <= k < N_C }
" */"
for (int c0 = 0; c0 < N_R; c0 += 1)
  for (int c1 = 0; c1 < N_C; c1 += 1)
    for (int c2 = index_i_; c2 < index_ip1_; c2 += 1)
      spmv(c0, c2);
"/* I_tile: "
[N_R, N_C, index_i_, index_ip1_] -> { spmv[ii, kk, i, k, j] : N_R >= 9 and N_C >= 9 and i >= 8ii and 0 <= i <= 7 + 8ii and i < N_R and k >= 8kk and 0 <= k <= 7 + 8kk and k < N_C and index_i_ <= j < index_ip1_ }
" */"
if (N_R >= 9 && N_C >= 9)
  for (int c0 = 0; c0 <= (N_R - 1) / 8; c0 += 1)
    for (int c1 = 0; c1 <= (N_C - 1) / 8; c1 += 1)
      for (int c2 = 8 * c0; c2 <= min(N_R - 1, 8 * c0 + 7); c2 += 1)
        for (int c3 = 8 * c1; c3 <= min(N_C - 1, 8 * c1

In [3]:
# While Loop Example
code = """
#define INT_MAX 2147483647
#define S(i) int p=!stack_empty(s)
#define T(i) dfs_code()
#define U(i) (i)=INT_MAX

Idfs := [INT_MAX,p] -> {S[i]->[i,0]: 0 <= i < INT_MAX;
                        T[i]->[i,1]: 0 <= i < INT_MAX && p > 0;
                        U[i]->[i,2]: 0 <= i < INT_MAX && p = 0};
print(Idfs);
codegen(Idfs);
"""

from tools.system import run
run('/usr/local/bin/iscc', code, verbose=True)


Output:

[INT_MAX, p] -> { T[i] -> [i, 1] : p > 0 and 0 <= i < INT_MAX; S[i] -> [i, 0] : 0 <= i < INT_MAX; U[i] -> [i, 2] : p = 0 and 0 <= i < INT_MAX }
for (int c0 = 0; c0 < INT_MAX; c0 += 1) {
  S(c0);
  if (p >= 1) {
    T(c0);
  } else if (p == 0) {
    U(c0);
  }
}



('[INT_MAX, p] -> { T[i] -> [i, 1] : p > 0 and 0 <= i < INT_MAX; S[i] -> [i, 0] : 0 <= i < INT_MAX; U[i] -> [i, 2] : p = 0 and 0 <= i < INT_MAX }\nfor (int c0 = 0; c0 < INT_MAX; c0 += 1) {\n  S(c0);\n  if (p >= 1) {\n    T(c0);\n  } else if (p == 0) {\n    U(c0);\n  }\n}\n',
 '')

In [1]:
# Dense Matrix Multiply
code = """
I_dmm := [M,N,P] -> {dmm[i,j,k]: 0 <= i < M && 0 <= j < P && 0 <= k < N};
print(I_dmm);
codegen(I_dmm);

I_tile := [M,N,P] -> {dmm[i,j,k] -> [floor(i/8),floor(j/8),i,j,k]: 0 <= i < M && 0 <= j < P && 0 <= k < N};
print(I_tile);
codegen(I_tile);
"""

from tools.system import run
run('/usr/local/bin/iscc', code, verbose=True)


Output:

[M, N, P] -> { dmm[i, j, k] : 0 <= i < M and 0 <= j < P and 0 <= k < N }
for (int c0 = 0; c0 < M; c0 += 1)
  for (int c1 = 0; c1 < P; c1 += 1)
    for (int c2 = 0; c2 < N; c2 += 1)
      dmm(c0, c1, c2);
[M, N, P] -> { dmm[i, j, k] -> [o0, o1, i, j, k] : 0 <= i < M and 0 <= j < P and 0 <= k < N and -7 + i <= 8o0 <= i and -7 + j <= 8o1 <= j }
for (int c0 = 0; c0 <= floord(M - 1, 8); c0 += 1)
  for (int c1 = 0; c1 <= floord(P - 1, 8); c1 += 1)
    for (int c2 = 8 * c0; c2 <= min(M - 1, 8 * c0 + 7); c2 += 1)
      for (int c3 = 8 * c1; c3 <= min(P - 1, 8 * c1 + 7); c3 += 1)
        for (int c4 = 0; c4 < N; c4 += 1)
          dmm(c2, c3, c4);



('[M, N, P] -> { dmm[i, j, k] : 0 <= i < M and 0 <= j < P and 0 <= k < N }\nfor (int c0 = 0; c0 < M; c0 += 1)\n  for (int c1 = 0; c1 < P; c1 += 1)\n    for (int c2 = 0; c2 < N; c2 += 1)\n      dmm(c0, c1, c2);\n[M, N, P] -> { dmm[i, j, k] -> [o0, o1, i, j, k] : 0 <= i < M and 0 <= j < P and 0 <= k < N and -7 + i <= 8o0 <= i and -7 + j <= 8o1 <= j }\nfor (int c0 = 0; c0 <= floord(M - 1, 8); c0 += 1)\n  for (int c1 = 0; c1 <= floord(P - 1, 8); c1 += 1)\n    for (int c2 = 8 * c0; c2 <= min(M - 1, 8 * c0 + 7); c2 += 1)\n      for (int c3 = 8 * c1; c3 <= min(P - 1, 8 * c1 + 7); c3 += 1)\n        for (int c4 = 0; c4 < N; c4 += 1)\n          dmm(c2, c3, c4);\n',
 '')

In [43]:
# Jacobi2D
code = """
D := [T,M,N] -> {S[t,i,j] : 1 <= t <= T && 1 <= i <= M && 1 <= j <= N};

R := [T,M,N] -> {
    S[t,i,j] -> A[t-1,i,j-1];
    S[t,i,j] -> A[t-1,i,j];
    S[t,i,j] -> A[t-1,i,j+1];
    S[t,i,j] -> A[t-1,i-1,j];
    S[t,i,j] -> A[t-1,i+1,j];
} * D;

W := [T,M,N] -> {
    S[t,i,j] -> A[t,i,j];
} * D;

Sch := [T,M,N] -> {S[t,i,j] -> [t,i,j]};
B4 := Sch << Sch;

RaW := (W . (R^-1)) * B4;
WaW := (W . (W^-1)) * B4;
WaR := (R . (W^-1)) * B4;

print "RaW deps:";
print RaW;

print "WaW deps:";
print WaW;

print "WaR deps:";
print WaR;
"""

from tools.system import run
run('/usr/local/bin/iscc', code, verbose=True)


Output:

"RaW deps:"
[T, M, N] -> { S[t, i, j] -> S[t' = 1 + t, i' = i, j' = 1 + j] : 0 < t < T and 0 < i <= M and 0 < j < N; S[t, i, j] -> S[t' = 1 + t, i' = 1 + i, j' = j] : 0 < t < T and 0 < i < M and 0 < j <= N; S[t, i, j] -> S[t' = 1 + t, i' = i, j' = j] : 0 < t < T and 0 < i <= M and 0 < j <= N; S[t, i, j] -> S[t' = 1 + t, i' = -1 + i, j' = j] : 0 < t < T and 2 <= i <= M and 0 < j <= N; S[t, i, j] -> S[t' = 1 + t, i' = i, j' = -1 + j] : 0 < t < T and 0 < i <= M and 2 <= j <= N }
"WaW deps:"
[T, M, N] -> {  }
"WaR deps:"
[T, M, N] -> {  }



('"RaW deps:"\n[T, M, N] -> { S[t, i, j] -> S[t\' = 1 + t, i\' = i, j\' = 1 + j] : 0 < t < T and 0 < i <= M and 0 < j < N; S[t, i, j] -> S[t\' = 1 + t, i\' = 1 + i, j\' = j] : 0 < t < T and 0 < i < M and 0 < j <= N; S[t, i, j] -> S[t\' = 1 + t, i\' = i, j\' = j] : 0 < t < T and 0 < i <= M and 0 < j <= N; S[t, i, j] -> S[t\' = 1 + t, i\' = -1 + i, j\' = j] : 0 < t < T and 2 <= i <= M and 0 < j <= N; S[t, i, j] -> S[t\' = 1 + t, i\' = i, j\' = -1 + j] : 0 < t < T and 0 < i <= M and 2 <= j <= N }\n"WaW deps:"\n[T, M, N] -> {  }\n"WaR deps:"\n[T, M, N] -> {  }\n',
 '')

In [75]:
# MFD-2D
code = """
# X-Direction
#flux step 1
Fx1 := [N]->{fx1[c,y,x] : 0<=c<4 and 0<=x<=N and 0<=y<N};

#flux step 2
Fx2 := [N]->{fx2[c,y,x] : 0<=c<4 and 0<=x<=N and 0<=y<N};

# Diff step 3
Dx := [N]->{dx[c,y,x] : 0<=c<4 and 0<=x<N and 0<=y<N};

# Y-Direction
#flux step 1
Fy1 := [N]->{fy1[c,y,x] : 0<=c<4 and 0<=x<N and 0<=y<=N};

#flux step 2
Fy2 := [N]->{fy2[c,y,x] : 0<=c<4 and 0<=x<N and 0<=y<=N};

# Diff step 3
Dy := [N]->{dy[c,y,x] : 0<=c<4 and 0<=x<N and 0<=y<N};

# Statements
#statement(Fx1) := "Cx1[c,y,x] = (1./12.)*(Bin[c,y,x-2] + 7.0 * Bin[c,y,x-1] +
#                                          Bin[c,y,x] + Bin[c,y,x+1])";
#statement(Fx2) := "Cx2[c,y,x] = Cx1[c,y,x] * 2.0 * Cx1[2,y,x]";
#statement(Dx) := "Bout[c,y,x] += Cx2[c,y,+1] - Cx2[c,y,x]";
#statement(Fy1) := "Cy1[c,y,x] = (1./12.)*(Bin[c,y-2,x] + 7.0 * Bin[c,y-1,x] +
#                                          Bin[c,y,x] + Bin[c,y+1,x])";
#statement(Fy2) := "Cy2[c,y,x] = Cy1[c,y,x] * 2.0 * Cy1[3,y,x]";
#statement(Dy) := "Bout[c,y,x] += Cy2[c,y+1,x] - Cy2[c,y,x]";

D := Fx1 + Fx2 + Dx + Fy1 + Fy2 + Dy;

S := [N] -> {
    fx1[c,y,x] -> [0,c,y,x];
    fx2[c,y,x] -> [1,c,y,x];
    dx[c,y,x] -> [2,c,y,x];
    fy1[c,y,x] -> [3,c,y,x];
    fy2[c,y,x] -> [4,c,y,x];
    dy[c,y,x] -> [5,c,y,x];
};

R := [N] -> {
    fx1[c,y,x] -> Bin[c,y,x-2];
    fx1[c,y,x] -> Bin[c,y,x-1];
    fx1[c,y,x] -> Bin[c,y,x];
    fx1[c,y,x] -> Bin[c,y,x+1];
    
    fx2[c,y,x] -> Cx1[c,y,x];
    fx2[c,y,x] -> Cx1[2,y,x];
    
    dx[c,y,x] -> Cx2[c,y,x];
    dx[c,y,x] -> Cx2[2,y,x+1];
    
    fy1[c,y,x] -> Bin[c,y-2,x];
    fy1[c,y,x] -> Bin[c,y-1,x];
    fy1[c,y,x] -> Bin[c,y,x];
    fy1[c,y,x] -> Bin[c,y+1,x];
    
    fy2[c,y,x] -> Cy1[c,y,x];
    fy2[c,y,x] -> Cy1[3,y,x];
    
    dy[c,y,x] -> Cy2[c,y,x];
    dy[c,y,x] -> Cy2[2,y+1,x];
} * D;

W := [N] -> {
    fx1[c,y,x] -> Cx1[c,y,x];
    fx2[c,y,x] -> Cx2[c,y,x];
    
    fy1[c,y,x] -> Cy1[c,y,x];    
    fy2[c,y,x] -> Cy2[c,y,x];
    
    dx[c,y,x] -> Bout[c,y,x];    
    dy[c,y,x] -> Bout[c,y,x];
} * D;

codegen(S * D);

#B4 := S << S;
#RaW := (W . (R^-1)) * B4;
#print "RaW deps:";
#print RaW;
#WaW := (W . (W^-1)) * B4;
#print "WaW deps:";
#print WaW;
#WaR := (R . (W^-1)) * B4;
#print "WaR deps:";
#print WaR;
"""

from tools.system import run
run('/usr/local/bin/iscc', code, verbose=True)


Output:

{
  for (int c1 = 0; c1 <= 3; c1 += 1)
    for (int c2 = 0; c2 < N; c2 += 1)
      for (int c3 = 0; c3 <= N; c3 += 1)
        fx1(c1, c2, c3);
  for (int c1 = 0; c1 <= 3; c1 += 1)
    for (int c2 = 0; c2 < N; c2 += 1)
      for (int c3 = 0; c3 <= N; c3 += 1)
        fx2(c1, c2, c3);
  for (int c1 = 0; c1 <= 3; c1 += 1)
    for (int c2 = 0; c2 < N; c2 += 1)
      for (int c3 = 0; c3 < N; c3 += 1)
        dx(c1, c2, c3);
  for (int c1 = 0; c1 <= 3; c1 += 1)
    for (int c2 = 0; c2 <= N; c2 += 1)
      for (int c3 = 0; c3 < N; c3 += 1)
        fy1(c1, c2, c3);
  for (int c1 = 0; c1 <= 3; c1 += 1)
    for (int c2 = 0; c2 <= N; c2 += 1)
      for (int c3 = 0; c3 < N; c3 += 1)
        fy2(c1, c2, c3);
  for (int c1 = 0; c1 <= 3; c1 += 1)
    for (int c2 = 0; c2 < N; c2 += 1)
      for (int c3 = 0; c3 < N; c3 += 1)
        dy(c1, c2, c3);
}



('{\n  for (int c1 = 0; c1 <= 3; c1 += 1)\n    for (int c2 = 0; c2 < N; c2 += 1)\n      for (int c3 = 0; c3 <= N; c3 += 1)\n        fx1(c1, c2, c3);\n  for (int c1 = 0; c1 <= 3; c1 += 1)\n    for (int c2 = 0; c2 < N; c2 += 1)\n      for (int c3 = 0; c3 <= N; c3 += 1)\n        fx2(c1, c2, c3);\n  for (int c1 = 0; c1 <= 3; c1 += 1)\n    for (int c2 = 0; c2 < N; c2 += 1)\n      for (int c3 = 0; c3 < N; c3 += 1)\n        dx(c1, c2, c3);\n  for (int c1 = 0; c1 <= 3; c1 += 1)\n    for (int c2 = 0; c2 <= N; c2 += 1)\n      for (int c3 = 0; c3 < N; c3 += 1)\n        fy1(c1, c2, c3);\n  for (int c1 = 0; c1 <= 3; c1 += 1)\n    for (int c2 = 0; c2 <= N; c2 += 1)\n      for (int c3 = 0; c3 < N; c3 += 1)\n        fy2(c1, c2, c3);\n  for (int c1 = 0; c1 <= 3; c1 += 1)\n    for (int c2 = 0; c2 < N; c2 += 1)\n      for (int c3 = 0; c3 < N; c3 += 1)\n        dy(c1, c2, c3);\n}\n',
 '')

In [74]:
# T_compact-and-pad(T_tile(T_make-dense(I_csr)))
#define R 8
#define C 8
#define S(i) int m=index[(i)]
#define T(i) int n=index[(i)+1]
#define P(i,k,j) int c=col[(j)]
#define U(i,k,j) y[(i)]+=a[(j)]*x[(k)]
#define V(ii) int p=b_index[(ii)]
#define W(ii) int q=b_index[(ii)+1]
#define X(jj) int kk=b_col[(jj)]
#define Y(ii,jj,i,k,z) y[(i)]+=a_prime[(jj)*R+((i)-R*(ii))*C+((k)-C*b_col[(jj)])]*x[(k)]

#   for (int c0 = 0; c0 <= (N_R - 1)/8; c0 += 1) {  // ii
#     V(c0);
#     W(c0);
#     for (int c1 = p; c1 < q; c1 += 1) {           // jj
#       X(c1);
#       for (int c2 = 8*c0; c2 <= min(N_R-1, 8*c0+7); c2 += 1)
#         for (int c3 = 8 * b_col[c1]; c3 <= min(8 * b_col[c1] + 7, N_C - 1); c3 += 1)
#           Y(c0, c1, c2, c3, 0);
#   }  }
code = """
Icsr := [N_R,N_C,NNZ,index_i_,index_ip1_,col_j_] -> {
    spmv[i,k,j]: 0 <= i < N_R && 0 <= k < N_C && index_i_ <= j < index_ip1_ && index_i_ >= 0 && index_ip1_ <= NNZ && k = col_j_
};
print("// Icsr");
print(Icsr);
#codegen(Icsr);

#Tdense := [N_R,N_C,NNZ,index_i_,index_ip1_,col_j_] -> {
#    spmv[i,k,j] -> [i,k,j]: 0 <= k < N_C && k = col_j_;
#};
#Idense := Tdense * Icsr;
#print("// Idense");
#print(Idense);
#codegen(Idense);

#Itile := [T,N] -> { S1[t,i]->[floor(t/4), floor(i/4), t, i]: 1 <= t <= T && 1 <= i <= N };
#codegen(Itile);
Ttile := [N_R,N_C,NNZ,index_i_,index_ip1_,col_j_] -> {
    spmv[i,k,j] -> [floor(i/8), floor(k/8), i,k,j]: N_R > 8 && N_C > 8
};
Itile := Ttile * Icsr;
print("// Itile");
print(Itile);
codegen(Itile);

#Itile := [N_R,N_C,m,n,c] -> {
#         S[i]->[ii,i,0,0,0,0,0]: 0 <= i < N_R && exists ri: 0 <= ri < 8 && i = ii*8+ri;
#         T[i]->[ii,i,1,0,0,0,0]: 0 <= i < N_R && exists ri: 0 <= ri < 8 && i = ii*8+ri;
#         P[i,k,j]->[ii,i,2,kk,k,j,0]: 0 <= i < N_R && 0 <= k < N_C && m <= j < n &&
#           exists ri,ck: 0 <= ri < 8 && i = ii*8+ri && 0 <= ck < 8 && k = kk*8+ck;
#         U[i,k,j]->[ii,i,2,kk,k,j,1]: 0 <= i < N_R && 0 <= k < N_C && m <= j < n && k=c &&
#           exists ri,ck: 0 <= ri < 8 && i = ii*8+ri && 0 <= ck < 8 && k = kk*8+ck};
#print(Itile);
#codegen(Itile);

#Texec := {
#  spmv[ii,kk,i,k,j] -> spmv[ii,jj,i,k,j]: b_index(ii) <= jj < b_index(ii+1) &&
#                                          kk = b_col(jj) && b_index(i) <= NB && j = 0
#};
Tcomp := [N_R,N_C,m,n,c] -> {
  S[i] -> [ii,i,0,0,0,0,0]
};
Icomp := Tcomp * Itile;
print(Icomp);
codegen(Icomp);
"""

cmd = '/usr/local/bin/iscc'# + ' --help'
import subprocess as sub
proc = sub.Popen(cmd, shell=True, stdin=sub.PIPE, stdout=sub.PIPE, stderr=sub.PIPE)
data = proc.communicate(input=code.encode('ascii'))
out = data[0].decode()
err = data[1].decode()

if len(out) > 0:
    print("Output:")
    print("=========================================================================================================\n")
    print(out)

if len(err) > 0:
    print("Error:")
    print("=========================================================================================================\n")
    print(err)


Output:

"// Icsr"
[N_R, N_C, NNZ, index_i_, index_ip1_, col_j_] -> { spmv[i, k = col_j_, j] : index_i_ >= 0 and index_ip1_ <= NNZ and 0 <= col_j_ < N_C and 0 <= i < N_R and index_i_ <= j < index_ip1_ }
"// Itile"
[N_R, N_C, NNZ, index_i_, index_ip1_, col_j_] -> { spmv[i, k = col_j_, j] -> [o0, o1, i, col_j_, j] : N_R >= 9 and N_C >= 9 and index_i_ >= 0 and index_ip1_ <= NNZ and 0 <= col_j_ < N_C and 0 <= i < N_R and index_i_ <= j < index_ip1_ and -7 + i <= 8o0 <= i and -7 + col_j_ <= 8o1 <= col_j_ }
if (N_R >= 9 && N_C >= 9 && index_i_ >= 0 && NNZ >= index_ip1_ && N_C >= col_j_ + 1 && col_j_ >= 0)
  for (int c0 = 0; c0 <= (N_R - 1) / 8; c0 += 1)
    for (int c2 = 8 * c0; c2 <= min(N_R - 1, 8 * c0 + 7); c2 += 1)
      for (int c4 = index_i_; c4 < index_ip1_; c4 += 1)
        spmv(c2, col_j_, c4);
[N_R, N_C, NNZ, index_i_, index_ip1_, col_j_, m, n, c] -> {  }
{
}



In [1]:
code = """
# SpMV w/ DDD
#define lb_j(i) m=index(i)
#define ub_j(i) n=index(i+1)
#define spmv(i,j) y[(i)]+=A[(j)]*x[col[(j)]]
#Icsr := [N_R, m, n] -> {lb_j[i]: 0 <= i < N_R; ub_j[i]: 0 <= i < N_R; spmv[i,j] : 0 <= i < N_R && m <= j < n};
#Tfuse := [N_R, m, n] -> {
#    lb_j[i] -> [i,0,0,0];
#    ub_j[i] -> [i,1,0,1];
#    spmv[i,j] -> [i,2,j,2];
#};
#Ifuse := Tfuse * Icsr;
#print(Ifuse);
#codegen(Ifuse);

# SpMV w/ DDD
#define S(i) m=index[i]
#define T(i) n=index[i+1]
#define U(j) k=col[j]
#define V(i,j) y[(i)]+=A[(j)]*x[col[(j)]]
Icsr := [N_R,m, n] -> {S[i]: 0 <= i < N_R; T[i]: 0 <= i < N_R; U[i,j]: 0 <= i < N_R && m <= j < n};
#     V[i,j] -> [i,2,j,3];
Tfuse := [N_R, m, n] -> {
    S[i] -> [i,0,0,0];
    T[i] -> [i,1,0,1];
    U[i,j] -> [i,2,j,2];
};
Ifuse := Tfuse * Icsr;
print(Ifuse);
codegen(Ifuse);

# Jacobi1D
#define index(t,i) t*N+i
#define stencil(t,i) A[index(t,i)] = (A[index(t-1,i-1)] + A[index(t-1,i)] + A[index(t-1,i+1)]) / 3.0

I_jac := [T,N] -> { stencil[t,i] : 1<=t<=T and 1<=i<=N };
codegen(I_jac);

# Interchange
T_ichg := [T,N] -> {stencil[t,i] -> [i,t]};
I_ichg := T_ichg * I_jac;
codegen(I_ichg);

# Skew
T_skew := [T,N] -> { stencil[t,i] -> [t,i+t] };
I_skew := T_skew * I_jac;
codegen(I_skew);

# Split Op
I_1 := [T,N] -> {S_1[t,i_1] : 1 <= t <= T && 1 <= i_1 <= floor(N/3)};
I_2 := [T,N] -> {S_2[t,i_2] : 1 <= t <= T && floor(N/3)+1 <= i_2 <= 2* floor(N/3)};
I_3 := [T,N] -> {S_3[t,i_3] : 1 <= t <= T && 2 * floor(N/3) + 1 <= i_3 <= N};
#codegen(I_1);
#codegen(I_2);
#codegen(I_3);

# Composition
#I_u := I_1 + I_2 + I_3;
#I_u := [T,N] -> {S_1[0,t,i_1] : 1 <= t <= T && 1 <= i_1 <= floor(N/3);
#                 S_2[1,t,i_2] : 1 <= t <= T && floor(N/3)+1 <= i_2 <= 2* floor(N/3);
#                 S_3[2,t,i_3] : 1 <= t <= T && 2 * floor(N/3) + 1 <= i_3 <= N};
I_u := [T,N] -> {S_1[t,i_1] : 1 <= t <= T && 1 <= i_1 <= floor(N/3);
                 S_2[t,i_2] : 1 <= t <= T && floor(N/3)+1 <= i_2 <= 2* floor(N/3);
                 S_3[t,i_3] : 1 <= t <= T && 2 * floor(N/3) + 1 <= i_3 <= N};
#print(I_u);
#codegen(I_u);

# Fusion
#I_f := fuse(I_1, I_2, I_3);
T_f := [T,N] -> {
    S_1[t,i_1] -> [t,1,i_1];
    S_2[t,i_2] -> [t,2,i_2];
    S_3[t,i_3] -> [t,3,i_3];
};
I_f := T_f * I_u;
print(I_f);
codegen(I_f);


# Loop Unrolling (f=4)
I_jam := [T,N] -> {S1[t,i]->[t,i,0] : 1 <= t <= T && exists a : 1 <= i <= N && i = 4a + 1;
                   S2[t,i]->[t,i,1] : 1 <= t <= T && exists a : 1 <= i <= N && i = 4a + 1;
                   S3[t,i]->[t,i,2] : 1 <= t <= T && exists a : 1 <= i <= N && i = 4a + 1;
                   S4[t,i]->[t,i,3] : 1 <= t <= T && exists a : 1 <= i <= N && i = 4a + 1};
print(I_jam);
codegen(I_jam);

# Tiling (4x4)
# S1[t,i] -> [floor(i/4)]
#Itile := [T,N] -> {
#    S1[tt,ti,t,i]: 1 <= t <= T && 1 <= i <= N && exists rt,ri: 1 <= rt <= 4 &&
#                       t = tt*4+rt && 1 <= ri <= 4 && i = ti*4+ri && T > 4 && N > 4};
# && T > 4 && N > 4
Itile := [T,N] -> { S1[t,i]->[floor(t/4), floor(i/4), t, i]: 1 <= t <= T && 1 <= i <= N };
codegen(Itile);

Ifuse := [T,N] -> {
    S0[t] -> [t,0,0]: 1 <= t <= T;
    S1[t,i] -> [t,1,i]: 1 <= t <= T && 1 <= i <= N
};
print(Ifuse);
codegen(Ifuse);
"""

cmd = '/usr/local/bin/iscc'# + ' --help'
from tools.system import run
run(cmd, code, verbose=True)


Output:

[N_R, m, n] -> { S[i] -> [i, 0, 0, 0] : 0 <= i < N_R; T[i] -> [i, 1, 0, 1] : 0 <= i < N_R; U[i, j] -> [i, 2, j, 2] : 0 <= i < N_R and m <= j < n }
for (int c0 = 0; c0 < N_R; c0 += 1) {
  S(c0);
  T(c0);
  for (int c2 = m; c2 < n; c2 += 1)
    U(c0, c2);
}
for (int c0 = 1; c0 <= T; c0 += 1)
  for (int c1 = 1; c1 <= N; c1 += 1)
    stencil(c0, c1);
for (int c0 = 1; c0 <= N; c0 += 1)
  for (int c1 = 1; c1 <= T; c1 += 1)
    stencil(c1, c0);
for (int c0 = 1; c0 <= T; c0 += 1)
  for (int c1 = c0 + 1; c1 <= N + c0; c1 += 1)
    stencil(c0, -c0 + c1);
[T, N] -> { S_3[t, i_3] -> [t, 3, i_3] : 0 < t <= T and i_3 <= N and 2*floor((N)/3) < i_3; S_2[t, i_2] -> [t, 2, i_2] : 0 < t <= T and 3i_2 > N and 2*floor((N)/3) >= i_2; S_1[t, i_1] -> [t, 1, i_1] : 0 < t <= T and i_1 > 0 and 3i_1 <= N }
for (int c0 = 1; c0 <= T; c0 += 1) {
  for (int c2 = 1; c2 <= floord(N, 3); c2 += 1)
    S_1(c0, c2);
  for (int c2 = floord(N, 3) + 1; c2 <= 2 * floord(N, 3); c2 += 1)
    S_2(c0, c2);
  for (int c2 =

('[N_R, m, n] -> { S[i] -> [i, 0, 0, 0] : 0 <= i < N_R; T[i] -> [i, 1, 0, 1] : 0 <= i < N_R; U[i, j] -> [i, 2, j, 2] : 0 <= i < N_R and m <= j < n }\nfor (int c0 = 0; c0 < N_R; c0 += 1) {\n  S(c0);\n  T(c0);\n  for (int c2 = m; c2 < n; c2 += 1)\n    U(c0, c2);\n}\nfor (int c0 = 1; c0 <= T; c0 += 1)\n  for (int c1 = 1; c1 <= N; c1 += 1)\n    stencil(c0, c1);\nfor (int c0 = 1; c0 <= N; c0 += 1)\n  for (int c1 = 1; c1 <= T; c1 += 1)\n    stencil(c1, c0);\nfor (int c0 = 1; c0 <= T; c0 += 1)\n  for (int c1 = c0 + 1; c1 <= N + c0; c1 += 1)\n    stencil(c0, -c0 + c1);\n[T, N] -> { S_3[t, i_3] -> [t, 3, i_3] : 0 < t <= T and i_3 <= N and 2*floor((N)/3) < i_3; S_2[t, i_2] -> [t, 2, i_2] : 0 < t <= T and 3i_2 > N and 2*floor((N)/3) >= i_2; S_1[t, i_1] -> [t, 1, i_1] : 0 < t <= T and i_1 > 0 and 3i_1 <= N }\nfor (int c0 = 1; c0 <= T; c0 += 1) {\n  for (int c2 = 1; c2 <= floord(N, 3); c2 += 1)\n    S_1(c0, c2);\n  for (int c2 = floord(N, 3) + 1; c2 <= 2 * floord(N, 3); c2 += 1)\n    S_2(c0, c2);\n

In [1]:
code = """
# SpMV w/ DDD
# print("#define S(i) L=rp[(i)]");
# print("#define T(i) U=rp[(i)+1]");
# print("#define U(n) k=col[(n)]");
# print("#define V(i,j) y[(i)]+=A[(j)]*x[col[(j)]]");
# Icsr := [N,L,U] -> {S[i]: 0 <= i < N; T[i]: 0 <= i < N; U[i,j]: 0 <= i < N && L <= j < U};
# Tfuse := [N,L,U] -> {
#     S[i] -> [i,0,0,0];
#     T[i] -> [i,1,0,1];
#     U[i,j] -> [i,2,j,2];
# };
# Ifuse := Tfuse * Icsr;
# codegen(Ifuse);

# Separating guards and statements into pairs
print("#define G1(n,i) c=((n) >= rp[(i)+1])");
print("#define S1(n,i) (rp[(i)+1]=(n)+1)");
print("#define G2(n,i) c=(rp[(i)] > rp[(i)+1])");
print("#define S2(n,i) (rp[(i)+1]=rp[(i)]");
#
#Irp1 := [NNZ,row_n_,c] -> {G1[n,i]: 0<=n<NNZ&&i=row_n_; S1[n,i]: 0<=n<NNZ&&i=row_n_&&c>0};
#Irp2 := [NNZ,row_n_,c] -> {G2[n,i]: 0<=n<NNZ&&i=row_n_; S2[n,i]: 0<=n<NNZ&&i=row_n_&&c>0};
#Trp1 := [NNZ,row_n_,c] -> {G1[n,i] -> [n,i,0]; S1[n,i] -> [n,i,1]};
#Trp2 := [NNZ,row_n_,c] -> {G2[n,i] -> [n,i,0]; S2[n,i] -> [n,i,1]};
#codegen(Trp1 * Irp1);
#codegen(Trp2 * Irp2);
#
Irp := [NNZ,row_n_,c] -> {
  G1[n,i]: 0<=n<NNZ&&i=row_n_; S1[n,i]: 0<=n<NNZ&&i=row_n_&&c>0;
  G2[n,i]: 0<=n<NNZ&&i=row_n_; S2[n,i]: 0<=n<NNZ&&i=row_n_&&c>0
};
Trp := [NNZ,row_n_,c] -> {G1[n,i] -> [n,i,0]; S1[n,i] -> [n,i,1];
                          G2[n,i] -> [n,i,2]; S2[n,i] -> [n,i,3]};
codegen(Trp * Irp);
"""

cmd = '/usr/local/bin/iscc'# + ' --help'
from tools.system import run
run(cmd, code, verbose=True)


Error:

/bin/sh: 1: /usr/local/bin/iscc: not found



('', '/bin/sh: 1: /usr/local/bin/iscc: not found\n')

In [1]:
code = """
# Jacobi2D
#define index(t,i) t*N+i
#define stencil(t,i) A[index(t,i)] = (A[index(t-1,i-1)] + A[index(t-1,i)] + A[index(t-1,i+1)]) / 3.0

I_jac := [T,M,N] -> { stencil[t,i,j] : 1<=t<=T and 1<=i<=M and 1<=j<=N };
codegen(I_jac);

# Loop Unrolling (f=2)
I_jam := [T,M,N] -> {stencil1[t,i]->[t,0,i,j] : exists a : 1 <= t <= T && t=2a+1 && 1 <= i <= M && 1 <= j <= N;
                     stencil2[t,i]->[t,1,i,j] : exists a : 1 <= t <= T && t=2a+1 && 1 <= i <= M && 1 <= j <= N};
print(I_jam);
codegen(I_jam);

# Tiling (4x4)
# S1[t,i] -> [floor(i/4)]
#Itile := [T,N] -> {
#    S1[tt,ti,t,i]: 1 <= t <= T && 1 <= i <= N && exists rt,ri: 1 <= rt <= 4 &&
#                       t = tt*4+rt && 1 <= ri <= 4 && i = ti*4+ri && T > 4 && N > 4};
# && T > 4 && N > 4
I_tile := [T,M,N] -> { stencil[t,i,j]->[t,floor(i/8), floor(j/8), i, j]: 1 <= t <= T && 1 <= i <= M && 1 <= j <= N};
codegen(I_tile);

#Ifuse := [T,N] -> {
#    S0[t] -> [t,0,0]: 1 <= t <= T;
#    S1[t,i] -> [t,1,i]: 1 <= t <= T && 1 <= i <= N
#};
#print(Ifuse);
#codegen(Ifuse);
"""

cmd = '/usr/local/bin/iscc'# + ' --help'
from tools.system import run
run(cmd, code, verbose=True)


Output:

for (int c0 = 1; c0 <= T; c0 += 1)
  for (int c1 = 1; c1 <= M; c1 += 1)
    for (int c2 = 1; c2 <= N; c2 += 1)
      stencil(c0, c1, c2);
[T, M, N] -> { stencil1[t, i] -> [t, 0, i, j] : 2*floor((-1 + t)/2) = -1 + t and 0 < t <= T and 0 < i <= M and 0 < j <= N; stencil2[t, i] -> [t, 1, i, j] : 2*floor((-1 + t)/2) = -1 + t and 0 < t <= T and 0 < i <= M and 0 < j <= N }
for (int c0 = 1; c0 <= T; c0 += 2) {
  for (int c2 = 1; c2 <= M; c2 += 1)
    for (int c3 = 1; c3 <= N; c3 += 1)
      stencil1(c0, c2);
  for (int c2 = 1; c2 <= M; c2 += 1)
    for (int c3 = 1; c3 <= N; c3 += 1)
      stencil2(c0, c2);
}
for (int c0 = 1; c0 <= T; c0 += 1)
  for (int c1 = 0; c1 <= floord(M, 8); c1 += 1)
    for (int c2 = 0; c2 <= floord(N, 8); c2 += 1)
      for (int c3 = max(1, 8 * c1); c3 <= min(M, 8 * c1 + 7); c3 += 1)
        for (int c4 = max(1, 8 * c2); c4 <= min(N, 8 * c2 + 7); c4 += 1)
          stencil(c0, c3, c4);



('for (int c0 = 1; c0 <= T; c0 += 1)\n  for (int c1 = 1; c1 <= M; c1 += 1)\n    for (int c2 = 1; c2 <= N; c2 += 1)\n      stencil(c0, c1, c2);\n[T, M, N] -> { stencil1[t, i] -> [t, 0, i, j] : 2*floor((-1 + t)/2) = -1 + t and 0 < t <= T and 0 < i <= M and 0 < j <= N; stencil2[t, i] -> [t, 1, i, j] : 2*floor((-1 + t)/2) = -1 + t and 0 < t <= T and 0 < i <= M and 0 < j <= N }\nfor (int c0 = 1; c0 <= T; c0 += 2) {\n  for (int c2 = 1; c2 <= M; c2 += 1)\n    for (int c3 = 1; c3 <= N; c3 += 1)\n      stencil1(c0, c2);\n  for (int c2 = 1; c2 <= M; c2 += 1)\n    for (int c3 = 1; c3 <= N; c3 += 1)\n      stencil2(c0, c2);\n}\nfor (int c0 = 1; c0 <= T; c0 += 1)\n  for (int c1 = 0; c1 <= floord(M, 8); c1 += 1)\n    for (int c2 = 0; c2 <= floord(N, 8); c2 += 1)\n      for (int c3 = max(1, 8 * c1); c3 <= min(M, 8 * c1 + 7); c3 += 1)\n        for (int c4 = max(1, 8 * c2); c4 <= min(N, 8 * c2 + 7); c4 += 1)\n          stencil(c0, c3, c4);\n',
 '')

In [10]:
code = """
# Tcoo -> Thicoo
Ihi := [L,NNZ,NB,bind1_b_,bind2_b_,bind3_b_,eind1_m_,eind2_m_,eind3_m_,bptr_b_,bptr_bp1_,crd1_m_,crd2_m_,crd3_m_] -> {
    krp[b, bi, bj, bk, m, ei, ej, ek, i, j, k, l] : bi - bind1_b_ = 0 && bj - bind2_b_ = 0 &&
    bk - bind3_b_ = 0 && ei - eind1_m_ = 0 && ej - eind2_m_ = 0 && ek - eind3_m_ = 0 &&
    8*bi + ei - i = 0 && 8*bj + ej - j = 0 && 8*bk + ek - k = 0 && b >= 0 && m >= 0 && l >= 0 &&
    bptr_b_ >= 0 && m - bptr_b_ >= 0 && L - 1 >= 0 && NNZ - bptr_bp1_ >= 0 && NNZ - 2 >= 0 && -b + NB - 1 >= 0 &&
    -m + NNZ - 1 >= 0 && -m + bptr_bp1_ - 1 >= 0 && -l + L - 1 >= 0 &&
    i-crd1_m_=0&&j-crd2_m_=0&&k-crd3_m_=0 };
print(Ihi);
codegen(Ihi);
"""

cmd = '/usr/local/bin/iscc'# + ' --help'
from tools.system import run
run(cmd, code, verbose=True)


Output:

[L, NNZ, NB, bind1_b_, bind2_b_, bind3_b_, eind1_m_, eind2_m_, eind3_m_, bptr_b_, bptr_bp1_, crd1_m_, crd2_m_, crd3_m_] -> { krp[b, bi = bind1_b_, bj = bind2_b_, bk = bind3_b_, m, ei = eind1_m_, ej = eind2_m_, ek = eind3_m_, i = 8bind1_b_ + eind1_m_, j = 8bind2_b_ + eind2_m_, k = 8bind3_b_ + eind3_m_, l] : crd1_m_ = 8bind1_b_ + eind1_m_ and crd2_m_ = 8bind2_b_ + eind2_m_ and crd3_m_ = 8bind3_b_ + eind3_m_ and L > 0 and NNZ >= 2 and bptr_b_ >= 0 and bptr_bp1_ <= NNZ and 0 <= b < NB and m >= bptr_b_ and 0 <= m < bptr_bp1_ and m < NNZ and 0 <= l < L }
if (NNZ >= 2 && bptr_b_ >= 0 && NNZ >= bptr_bp1_ && crd1_m_ == 8 * bind1_b_ + eind1_m_ && crd2_m_ == 8 * bind2_b_ + eind2_m_ && crd3_m_ == 8 * bind3_b_ + eind3_m_)
  for (int c0 = 0; c0 < NB; c0 += 1)
    for (int c4 = bptr_b_; c4 < bptr_bp1_; c4 += 1)
      for (int c11 = 0; c11 < L; c11 += 1)
        krp(c0, bind1_b_, bind2_b_, bind3_b_, c4, eind1_m_, eind2_m_, eind3_m_, 8 * bind1_b_ + eind1_m_, 8 * bind2_b_ + eind2_m_, 8 * bind3_

('[L, NNZ, NB, bind1_b_, bind2_b_, bind3_b_, eind1_m_, eind2_m_, eind3_m_, bptr_b_, bptr_bp1_, crd1_m_, crd2_m_, crd3_m_] -> { krp[b, bi = bind1_b_, bj = bind2_b_, bk = bind3_b_, m, ei = eind1_m_, ej = eind2_m_, ek = eind3_m_, i = 8bind1_b_ + eind1_m_, j = 8bind2_b_ + eind2_m_, k = 8bind3_b_ + eind3_m_, l] : crd1_m_ = 8bind1_b_ + eind1_m_ and crd2_m_ = 8bind2_b_ + eind2_m_ and crd3_m_ = 8bind3_b_ + eind3_m_ and L > 0 and NNZ >= 2 and bptr_b_ >= 0 and bptr_bp1_ <= NNZ and 0 <= b < NB and m >= bptr_b_ and 0 <= m < bptr_bp1_ and m < NNZ and 0 <= l < L }\nif (NNZ >= 2 && bptr_b_ >= 0 && NNZ >= bptr_bp1_ && crd1_m_ == 8 * bind1_b_ + eind1_m_ && crd2_m_ == 8 * bind2_b_ + eind2_m_ && crd3_m_ == 8 * bind3_b_ + eind3_m_)\n  for (int c0 = 0; c0 < NB; c0 += 1)\n    for (int c4 = bptr_b_; c4 < bptr_bp1_; c4 += 1)\n      for (int c11 = 0; c11 < L; c11 += 1)\n        krp(c0, bind1_b_, bind2_b_, bind3_b_, c4, eind1_m_, eind2_m_, eind3_m_, 8 * bind1_b_ + eind1_m_, 8 * bind2_b_ + eind2_m_, 8 * bind3_b_

In [11]:
code = """
# Tcoo -> Tcsf
Icsf := [L,NNZ,crd1_m_,ind1_p_,crd2_m_,ind2_q_,crd3_m_,ind3_m_,off1_0,off1_1,off2_p_,off2_pp1_,off3_q_,off3_qp1_] -> {
     krp[p,i,q,j,m,k,l]: i - crd1_m_ = 0 && i - ind1_p_ = 0 && j - crd2_m_ = 0 && j - ind2_q_ = 0 &&
                         k - crd3_m_ = 0 && k - ind3_m_ = 0 && m >= 0 && l >= 0 && off3_q_ >= 0 &&
                         p - off1_0 >= 0 && q - off2_p_ >= 0 && m - off3_q_ >= 0 && L - 1 >= 0 &&
                         NNZ - off3_qp1_ >= 0 && NNZ - 2 >= 0 && -p + off1_1 - 1 >= 0 && -q + off2_pp1_ - 1 >= 0 &&
                         -m + NNZ - 1 >= 0 && -m + off3_qp1_ - 1 >= 0 && -l + L - 1 >= 0 };
print(Icsf);
codegen(Icsf);
"""

cmd = '/usr/local/bin/iscc'# + ' --help'
from tools.system import run
run(cmd, code, verbose=True)


Output:

[L, NNZ, crd1_m_, ind1_p_, crd2_m_, ind2_q_, crd3_m_, ind3_m_, off1_0, off1_1, off2_p_, off2_pp1_, off3_q_, off3_qp1_] -> { krp[p, i = crd1_m_, q, j = crd2_m_, m, k = crd3_m_, l] : ind1_p_ = crd1_m_ and ind2_q_ = crd2_m_ and ind3_m_ = crd3_m_ and L > 0 and NNZ >= 2 and off3_q_ >= 0 and off3_qp1_ <= NNZ and off1_0 <= p < off1_1 and off2_p_ <= q < off2_pp1_ and m >= off3_q_ and 0 <= m < off3_qp1_ and m < NNZ and 0 <= l < L }
if (NNZ >= 2 && ind1_p_ == crd1_m_ && ind2_q_ == crd2_m_ && ind3_m_ == crd3_m_ && off3_q_ >= 0 && NNZ >= off3_qp1_)
  for (int c0 = off1_0; c0 < off1_1; c0 += 1)
    for (int c2 = off2_p_; c2 < off2_pp1_; c2 += 1)
      for (int c4 = off3_q_; c4 < off3_qp1_; c4 += 1)
        for (int c6 = 0; c6 < L; c6 += 1)
          krp(c0, crd1_m_, c2, crd2_m_, c4, crd3_m_, c6);



('[L, NNZ, crd1_m_, ind1_p_, crd2_m_, ind2_q_, crd3_m_, ind3_m_, off1_0, off1_1, off2_p_, off2_pp1_, off3_q_, off3_qp1_] -> { krp[p, i = crd1_m_, q, j = crd2_m_, m, k = crd3_m_, l] : ind1_p_ = crd1_m_ and ind2_q_ = crd2_m_ and ind3_m_ = crd3_m_ and L > 0 and NNZ >= 2 and off3_q_ >= 0 and off3_qp1_ <= NNZ and off1_0 <= p < off1_1 and off2_p_ <= q < off2_pp1_ and m >= off3_q_ and 0 <= m < off3_qp1_ and m < NNZ and 0 <= l < L }\nif (NNZ >= 2 && ind1_p_ == crd1_m_ && ind2_q_ == crd2_m_ && ind3_m_ == crd3_m_ && off3_q_ >= 0 && NNZ >= off3_qp1_)\n  for (int c0 = off1_0; c0 < off1_1; c0 += 1)\n    for (int c2 = off2_p_; c2 < off2_pp1_; c2 += 1)\n      for (int c4 = off3_q_; c4 < off3_qp1_; c4 += 1)\n        for (int c6 = 0; c6 < L; c6 += 1)\n          krp(c0, crd1_m_, c2, crd2_m_, c4, crd3_m_, c6);\n',
 '')

In [9]:
code = """
# Overlapped Tiling
Sfx := [N]->{fx[y,x] : 0<=y<N and 0<=x<=N};
Tot := [N] -> {fx[y,x] -> [ty,tx,1,y+1,x,0]: exists ry,rx: 0<=ry<8 and y+1=ty*8+ry and
                            -1<=rx<8 and x=tx*8+rx };
Sot := Tot * Sfx;                         
print(Sot);
codegen(Sot);
"""

cmd = '/usr/local/bin/iscc'# + ' --help'
from tools.system import run
run(cmd, code, verbose=True)


Output:

[N] -> { fx[y, x] -> [ty, tx, 1, 1 + y, x, 0] : 0 <= y < N and 0 <= x <= N and -6 + y <= 8ty <= 1 + y and -7 + x <= 8tx <= 1 + x }
for (int c0 = 0; c0 <= floord(N, 8); c0 += 1)
  for (int c1 = 0; c1 <= (N + 1) / 8; c1 += 1)
    for (int c3 = max(1, 8 * c0); c3 <= min(N, 8 * c0 + 7); c3 += 1)
      for (int c4 = max(0, 8 * c1 - 1); c4 <= min(N, 8 * c1 + 7); c4 += 1)
        fx(c3 - 1, c4);



('[N] -> { fx[y, x] -> [ty, tx, 1, 1 + y, x, 0] : 0 <= y < N and 0 <= x <= N and -6 + y <= 8ty <= 1 + y and -7 + x <= 8tx <= 1 + x }\nfor (int c0 = 0; c0 <= floord(N, 8); c0 += 1)\n  for (int c1 = 0; c1 <= (N + 1) / 8; c1 += 1)\n    for (int c3 = max(1, 8 * c0); c3 <= min(N, 8 * c0 + 7); c3 += 1)\n      for (int c4 = max(0, 8 * c1 - 1); c4 <= min(N, 8 * c1 + 7); c4 += 1)\n        fx(c3 - 1, c4);\n',
 '')